In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):

    return movie

In [37]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load(wiki, kaggle, rating):
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    ratings = pd.read_csv(rating)
    kaggle_metadata = pd.read_csv(kaggle, low_memory=False)

    # Open and read the Wikipedia data JSON file.
    file = open(wiki)
    wiki_movies_raw = json.load(file)
    
    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies_raw = [data for data in wiki_movies_raw if 'Television series' not in data.keys()]

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    cleaned_movies = [clean_movie(movie) for movie in wiki_movies_raw]
    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(cleaned_movies)


    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        unique_imdb = []
        wiki_movies = []
        for data in cleaned_movies:
            if 'imdb_link' in data.keys():
                imdb_id = re.findall(r'(tt\d{7})', data['imdb_link'])[0]
                if imdb_id not in unique_imdb:
                    wiki_movies.append(data)
                    unique_imdb.append(imdb_id)
    except:
        print('Error Occurs while removing duplicates from data using IMDB')

    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_movies_df = pd.DataFrame(wiki_movies)
    wiki_movies_df['imdb_link'] = unique_imdb
    wiki_movies_df = wiki_movies_df[[col for col in wiki_movies_df.columns if not wiki_movies_df[col].isnull().values.all()]]

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    wiki_movies_df = wiki_movies_df[wiki_movies_df['Box office'].notnull()]
    box_office = wiki_movies_df['Box office'].values
    box_office = [data for data in box_office if isinstance(data, str)]
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office_func = lambda x:' '.join(x)
    box_office_data = box_office_func(box_office)
    
    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'(\d\d,\d\d\d | \d\d\d.\d\d)'
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'(\d,\d | \d.\d)'

    # 12. Add the parse_dollars function.
    def parse_dollars():
        pass
    
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    for i in range(wiki_movies_df.shape[0]):
        try:
            val = wiki_movies_df.loc[i,'Box office']
            if not isinstance(val, str):
                val = val[0]
            try:
                wiki_movies_df.loc[i,'Box office'] = re.findall(form_one, val)[0]
            except:
                try:
                    wiki_movies_df.loc[i,'Box office'] = re.findall(form_two, val)[0]
                except:
                    wiki_movies_df.loc[i,'Box office'] = val
        except:
            pass
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    for i in range(wiki_movies_df.shape[0]):
        try:
            val = wiki_movies_df.loc[i,'Budget']
            if not isinstance(val, str):
                val = val[0]
            wiki_movies_df.loc[i,'Budget'] = re.findall(r'([0-9]+)', val)[0]
        except:
            pass

    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    for i in range(wiki_movies_df.shape[0]):
        try:
            val = wiki_movies_df.loc[i,'Release date']
            if not isinstance(val, str):
                val = val[0]
            wiki_movies_df.loc[i,'Release date'] = val
        except:
            pass

    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    for i in range(wiki_movies_df.shape[0]):
        try:
            val = wiki_movies_df.loc[i,'Running time']
            if not isinstance(val, str):
                val = val[0]
            wiki_movies_df.loc[i,'Running time'] = re.findall(r'([0-9]+)', val)[0]
        except:
            pass
    
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_movies_df, kaggle_metadata, ratings 

In [38]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = 'Resources'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [39]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load(wiki_file, kaggle_file, ratings_file)

In [40]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [41]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,tt0098987,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,tt0098994,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,tt0099005,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,tt0099012,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,tt0099018,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Traditional',
 'Mandarin',
 'Released',
 'Rec